In [1]:
import os
import time
from datetime import datetime, timedelta

In [2]:
os.add_dll_directory('C:\\Program Files\\IBM\\SQLLIB\\BIN')
# conectar a la base de datos IBM Db2
import ibm_db

In [3]:
from queries import get_academic_history, get_socioeconomico, tasa_reprobacion, dificultad_materia
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
def format_time(seconds):
    """Convierte segundos a formato HH:MM:SS"""
    return str(timedelta(seconds=int(seconds)))

def print_progress_bar(current, total, bar_length=30):
    """Imprime una barra de progreso"""
    percent = float(current) * 100 / total
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    return f'[{bar}] {percent:.1f}% ({current}/{total})'

In [5]:
print("="*80)
print("INICIO DE DESCARGA DE DATOS ACADÉMICOS")
print("="*80)
print(f"Fecha y hora de inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

INICIO DE DESCARGA DE DATOS ACADÉMICOS
Fecha y hora de inicio: 2025-10-08 08:18:40


In [6]:
# Tiempo de inicio total
start_time_total = time.time()

print("\n📡 Conectando con la base de datos...")
start_time_conn = time.time()


📡 Conectando con la base de datos...


In [7]:
conn_str = (
    'DATABASE=' + os.getenv('DATABASE') + 
    ';HOSTNAME=' + os.getenv('HOSTNAME') + 
    ';PORT=' + os.getenv('PORT') + 
    ';PROTOCOL=TCPIP;UID=' + os.getenv('USERNAME_DB') + 
    ';PWD=' + os.getenv('PASSWORD_DB') + ';'
)

print(repr(conn_str))
conn = ibm_db.connect(conn_str, '', '')

'DATABASE=SAAC;HOSTNAME=192.168.254.53;PORT=50000;PROTOCOL=TCPIP;UID=espol;PWD=espol123;'


In [8]:
end_time_conn = time.time()
connection_time = end_time_conn - start_time_conn

if conn:
    print(f"✅ Conexión exitosa - Tiempo: {connection_time:.2f} segundos")
else:
    print("❌ Error al conectar")
    exit(1)

✅ Conexión exitosa - Tiempo: 33.90 segundos


In [9]:
# Cargar datos de materias
print("\n📁 Cargando lista de materias...")
start_time_load = time.time()
df_materias = pd.read_csv("./data/materias/dificultad_materia.csv")
end_time_load = time.time()
load_time = end_time_load - start_time_load

print(f"✅ Materias cargadas - Tiempo: {load_time:.2f} segundos")
print(f"📊 Total de materias en archivo: {len(df_materias)}")


📁 Cargando lista de materias...
✅ Materias cargadas - Tiempo: 0.00 segundos
📊 Total de materias en archivo: 1926


In [10]:
# # Filtrar materias a procesar
# list_not_grades = ["PORTAFOLIO"]
# materias_to_process = []

# for i in df_materias.iterrows():
#     if i[1]["MATERIA_x"] in list_not_grades or "MATERIA INTEGRADORA" in i[1]["MATERIA_x"]:
#         continue
#     materias_to_process.append(i[1])
materias_to_process = df_materias[df_materias["CLASIFMATERIA"] == "G"]

In [11]:
total_materias = len(materias_to_process)
print(f"🎯 Materias a procesar: {total_materias}")

# Parámetros de configuración
year = "2021"
semester = "2S"
print(f"📅 Período: {year} - {semester}")

🎯 Materias a procesar: 1892
📅 Período: 2021 - 2S


In [12]:
print("\n" + "="*80)
print("INICIANDO PROCESAMIENTO DE MATERIAS")
print("="*80)

# Variables para estadísticas
total_students_processed = 0
failed_materias = []
processing_times = []
df_complete = pd.DataFrame()


INICIANDO PROCESAMIENTO DE MATERIAS


In [13]:
# Procesar cada materia
# for counter, materia_data in enumerate(materias_to_process, 1):
for counter, materia_data in materias_to_process.iterrows():
    counter += 1
    print(counter, materia_data)
    print(f"\n{'='*60}")
    print(f"PROCESANDO MATERIA {counter}/{total_materias}")
    print(f"{'='*60}")
    
    cod_materia = materia_data["CODIGOMATERIA"].strip()
    # materia_name = materia_data["MATERIA_x"]
    materia_name = materia_data["MATERIA"].strip()
    
    print(f"📚 Código: {cod_materia}")
    print(f"📖 Nombre: {materia_name}")
    print(f"⏱️  {print_progress_bar(counter-1, total_materias)}")
    
    # Tiempo de inicio para esta materia
    start_time_materia = time.time()
    
    try:
        # Query 1: Historial académico
        print(f"\n🔍 Ejecutando query de historial académico...")
        start_time_query1 = time.time()
        
        query = get_academic_history(cod_materia, year, semester)
        stmt_select = ibm_db.exec_immediate(conn, query)
        
        data_list = []
        result = ibm_db.fetch_assoc(stmt_select)
        while result:
            result["COD_MATERIA_ACAD_MO"] = result["COD_MATERIA_ACAD_MO"].strip()
            result["CODIGOMATERIA"] = cod_materia
            result["MATERIA"] = materia_name
            data_list.append(result)
            result = ibm_db.fetch_assoc(stmt_select)
        
        df = pd.DataFrame(data_list)
        end_time_query1 = time.time()
        query1_time = end_time_query1 - start_time_query1
        
        num_students = len(df)
        print(f"✅ Query 1 completada - Estudiantes encontrados: {num_students} - Tiempo: {query1_time:.2f}s")
        
        if num_students == 0:
            print(f"⚠️  No hay estudiantes para esta materia, saltando...")
            continue
        
        # # Query 2: Datos socioeconómicos
        # print(f"🔍 Ejecutando query socioeconómico...")
        # start_time_query2 = time.time()

        # query = get_socioeconomico(df["COD_ESTUDIANTE"].to_list())
        # stmt_select = ibm_db.exec_immediate(conn, query)

        # data_list = []
        # result = ibm_db.fetch_assoc(stmt_select)
        # while result:
        #     data_list.append(result)
        #     result = ibm_db.fetch_assoc(stmt_select)

        # df_socio = pd.DataFrame(data_list)
        # end_time_query2 = time.time()
        # query2_time = end_time_query2 - start_time_query2

        # socio_records = len(df_socio)
        socio_records = len(df)
        # print(f"✅ Query 2 completada - Registros socioeconómicos: {socio_records} - Tiempo: {query2_time:.2f}s")

        # if df_socio.empty:
        #     print(f"⚠️ No se encontraron datos socioeconómicos de {len(df["COD_ESTUDIANTE"])} estudiante/s. Se crearán columnas vacías.")
        #     df_socio = pd.DataFrame(columns=['CODESTUDIANTE']) # Asegura que la columna exista.


        # # Merge y guardado
        # print(f"🔗 Realizando merge y guardando archivo...")
        # start_time_merge = time.time()

        # df["COD_ESTUDIANTE"] = df["COD_ESTUDIANTE"].astype(str).str.strip()
        # df_socio["CODESTUDIANTE"] = df_socio["CODESTUDIANTE"].astype(str).str.strip()

        # df_merge = pd.merge(df, df_socio, left_on='COD_ESTUDIANTE', right_on="CODESTUDIANTE", how='left')

        # if "CODESTUDIANTE" in df_merge.columns:
        #     df_merge.drop("CODESTUDIANTE", axis=1, inplace=True)
            
        # df_complete = pd.concat([df_complete, df_merge], ignore_index=True)
        df_complete = pd.concat([df_complete, df], ignore_index=True)

        end_time_merge = time.time()
        # merge_time = end_time_merge - start_time_merge

        # Tiempo total para esta materia
        end_time_materia = time.time()
        materia_time = end_time_materia - start_time_materia
        processing_times.append(materia_time)
        
        
        # print(f"💾 Merge y guardado - Tiempo: {merge_time:.2f}s")
        # print(f"📊 Registros finales: {len(df_merge)}")
        
        # Estadísticas de la materia
        total_students_processed += num_students
        coverage = (socio_records / num_students * 100) if num_students > 0 else 0
        
        print(f"\n📈 RESUMEN MATERIA:")
        print(f"   • Estudiantes académicos: {num_students}")
        # print(f"   • Registros socioeconómicos: {socio_records}")
        print(f"   • Cobertura socioeconómica: {coverage:.1f}%")
        print(f"   • Query 1: {query1_time:.2f}s")
        # print(f"   • Query 2: {query2_time:.2f}s")
        # print(f"   • Merge/Guardado: {merge_time:.2f}s")
        print(f"   • Tiempo total materia: {format_time(materia_time)}")
        
        # Estimación de tiempo restante
        if len(processing_times) >= 3:  # Solo después de procesar al menos 3 materias
            avg_time_per_materia = sum(processing_times) / len(processing_times)
            remaining_materias = total_materias - counter
            estimated_remaining_time = avg_time_per_materia * remaining_materias
            
            print(f"⏰ Tiempo promedio por materia: {format_time(avg_time_per_materia)}")
            print(f"⏳ Tiempo estimado restante: {format_time(estimated_remaining_time)}")
            
            estimated_finish = datetime.now() + timedelta(seconds=estimated_remaining_time)
            print(f"🏁 Finalización estimada: {estimated_finish.strftime('%H:%M:%S')}")
        
        # if counter == 2:
        #     break
    except Exception as e:
        print(f"❌ ERROR procesando {cod_materia}: {str(e)}")
        failed_materias.append({
            'codigo': cod_materia,
            # 'nombre': materia_name,
            'error': str(e)
        })

1 CODIGOMATERIA                  ACUG1035
CLASIFMATERIA                         G
MATERIA          ACUICULTURA ORNAMENTAL
ANIO                               2020
TERMINO                              0S
DIFICULTAD                         8,52
Name: 0, dtype: object

PROCESANDO MATERIA 1/1892
📚 Código: ACUG1035
📖 Nombre: ACUICULTURA ORNAMENTAL
⏱️  [------------------------------] 0.0% (0/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 8 - Tiempo: 18.69s

📈 RESUMEN MATERIA:
   • Estudiantes académicos: 8
   • Cobertura socioeconómica: 100.0%
   • Query 1: 18.69s
   • Tiempo total materia: 0:00:18
2 CODIGOMATERIA                       ACUG1036
CLASIFMATERIA                              G
MATERIA          ANÁLISIS DE DATOS ACUÍCOLAS
ANIO                                    2020
TERMINO                                   0S
DIFICULTAD                              7,72
Name: 1, dtype: object

PROCESANDO MATERIA 2/1892
📚 Código: ACUG1036
📖 Nomb

In [14]:
df_complete.shape

(31286, 27)

In [15]:
filename = f"./data/materias/all_{year}_{semester}.csv"
df_complete.to_csv(filename, index=False)
print(f"✅ Archivo guardado: {filename}")

✅ Archivo guardado: ./data/materias/all_2021_2S.csv


In [16]:
# Cerrar conexión
print(f"\n🔌 Cerrando conexión a la base de datos...")
ibm_db.close(conn)

# Tiempo total
end_time_total = time.time()
total_time = end_time_total - start_time_total



🔌 Cerrando conexión a la base de datos...


In [17]:
print("\n" + "="*80)
print("RESUMEN FINAL DEL PROCESAMIENTO")
print("="*80)
print(f"🏁 Fecha y hora de finalización: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️  Tiempo total de ejecución: {format_time(total_time)}")
print(f"📊 Materias procesadas exitosamente: {total_materias - len(failed_materias)}/{total_materias}")
print(f"👥 Total estudiantes procesados: {total_students_processed:,}")



RESUMEN FINAL DEL PROCESAMIENTO
🏁 Fecha y hora de finalización: 2025-10-08 18:02:27
⏱️  Tiempo total de ejecución: 9:43:46
📊 Materias procesadas exitosamente: 1892/1892
👥 Total estudiantes procesados: 31,286


In [18]:
if processing_times:
    avg_time = sum(processing_times) / len(processing_times)
    min_time = min(processing_times)
    max_time = max(processing_times)
    
    print(f"\n📈 ESTADÍSTICAS DE TIEMPO:")
    print(f"   • Tiempo promedio por materia: {format_time(avg_time)}")
    print(f"   • Tiempo mínimo: {format_time(min_time)}")
    print(f"   • Tiempo máximo: {format_time(max_time)}")
    print(f"   • Velocidad promedio: {total_students_processed/total_time:.1f} estudiantes/segundo")


📈 ESTADÍSTICAS DE TIEMPO:
   • Tiempo promedio por materia: 0:00:31
   • Tiempo mínimo: 0:00:18
   • Tiempo máximo: 0:00:57
   • Velocidad promedio: 0.9 estudiantes/segundo


In [19]:
if failed_materias:
    print(f"\n❌ MATERIAS CON ERRORES ({len(failed_materias)}):")
    for failed in failed_materias:
        print(f"   • {failed['codigo']} - {failed['nombre'][:50]}")
        print(f"     Error: {failed['error']}")

❌ MATERIAS CON ERRORES (15):
   • ACUG1045 - PRODUCCIÓN ACUÍCOLA II
     Error: 'CODESTUDIANTE'
   • ALIG1033 - DISEÑO DE PLANTAS ALIMENTARIAS
     Error: 'CODESTUDIANTE'
   • CCPG1058 - SISTEMAS DE INFORMACIÓN APLICADOS A LOGÍSTICA
     Error: 'CODESTUDIANTE'
   • ESTG1056 - DATOS NO ESTRUCTURADOS
     Error: 'CODESTUDIANTE'
   • GEOG1027 - DISEÑO DE CAMPO GEOLÓGICO
     Error: 'CODESTUDIANTE'
   • GEOG1037 - INGENIERÍA GEOLÓGICA
     Error: 'CODESTUDIANTE'
   • INDG1061 - HSEQ EN LA INDUSTRIA HIDROCARBURÍFERA
     Error: 'CODESTUDIANTE'
   • MING1031 - PLANEAMIENTO Y DISEÑO MINERO
     Error: 'CODESTUDIANTE'
   • MTRG1027 - DISEÑO DE MATERIALES COMPUESTOS
     Error: 'CODESTUDIANTE'
   • MTRG1041 - NANOTECNOLOGÍA Y NANOMATERIALES
     Error: 'CODESTUDIANTE'
   • PETG1022 - INGENIERÍA DE PRODUCCIÓN II
     Error: 'CODESTUDIANTE'
   • PETG1023 - INGENIERÍA DE YACIMIENTOS I
     Error: 'CODESTUDIANTE'
   • PETG1024 - INGENIERÍA DE YACIMIENTOS II
     Error: 'CODESTUDIANTE'
   • PETG1028 - SIMULACIÓN DE RESERVORIOS
     Error: 'CODESTUDIANTE'
   • PETG1034 - EVALUACIÓN DE FORMACIONES II
     Error: 'CODESTUDIANTE'

In [20]:

print(f"\n✅ Procesamiento completado!")
print("="*80)


✅ Procesamiento completado!
